In [1]:
!pip install pandas
!pip install requests

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# International trading prices for commodities from Federal Reserve St. Louis

In [25]:
# food commodity 
idnames = {
    'bananas (U.S. Dollars per Metric Ton)': "PBANSOPUSDM",
    'olive oil (U.S. Dollars per Metric Ton)': "POLVOILUSDM",
    'sugar (U.S. Cents per Pound)': "PSUGAISAUSDM",
    'uranium (U.S. Dollars per Pound)': "PURANUSDM",
    'cotton (U.S. Cents per Pound)': "PCOTTINDUSDM",
    'orange (U.S. Dollars per Metric Ton)': "PORANGUSDM",
    'wheat (U.S. Dollars per Metric Ton)': "PWHEAMTUSDM",
    'aluminium (U.S. Dollars per Metric Ton)': "PALUMUSDM",
    'iron (U.S. Dollars per Metric Ton)': "PIORECRUSDM",
    'corn (U.S. Dollars per Metric Ton)': "PMAIZMTUSDM"
}

In [3]:
url = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id={}&scale=left&cosd=1980-01-01&coed=2017-06-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=0&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2009-06-01&line_index=1&transformation=lin&vintage_date=2018-10-18&revision_date=2018-10-18&nd=1980-01-01"

In [4]:
import requests 

def download_and_get_dataset(name, idname):
    r = requests.get(url.format(idname), allow_redirects=True)
    open(idname + '.csv', 'wb').write(r.content)
    df = pd.read_csv(idname + '.csv')
    df['value'] = df[idname]
    df['feature'] = name
    df['commodity'] = name.split(' ')[0]
    del df[idname]
    return df

In [5]:
import pandas as pd

data = {k: download_and_get_dataset(name=k, idname=v) for k, v in idnames.items()}

In [6]:
dfs = pd.concat([v for k, v in data.items()])
dfs.head(3)

,DATE,value,feature,commodity
0,1980-01-01,401.960839,bananas (U.S. Dollars per Metric Ton),bananas
1,1980-02-01,372.185964,bananas (U.S. Dollars per Metric Ton),bananas
2,1980-03-01,422.913536,bananas (U.S. Dollars per Metric Ton),bananas


## Save first dataset

In [7]:
dfs.to_csv('fsb_st_louis_commodities.csv', index=False)

# Import value and volume of Bananas from USDA

```
https://data.ers.usda.gov/reports.aspx?programArea=fruit&stat_year=2009&top=5&HardCopy=True&RowsPerPage=25&groupName=Noncitrus&commodityName=Bananas&ID=17851
```

In [8]:
def assign_feature(idx):
    if idx < 10:
        return 'dried bananas'
    elif idx < 20:
        return 'fresh bananas'
    elif idx < 30:
        return 'frozen bananas'
    else:
        return 'preserved bananas'

In [9]:
data = []
filename = './us_import_value_of_bananas.csv'
with open(filename, 'r') as f:
    for line in f.readlines():
        data.append(line.split('\t'))

data_filtered = []
for idx, item in enumerate(data):
    if item[-1]=='\n':
        item.pop(-1)
    item[-1] = item[-1].strip()
    if idx!=0:
        item = [item[0]] + [float(e.replace(',', '')) if e!='NA' else None for e in item[1:]]
    data_filtered.append(item)
    
df = pd.DataFrame(data_filtered[1:], columns=data_filtered[0])    
df['feature'] = df.index.map(lambda idx: assign_feature(idx))       
df['commodity'] = 'banana'
df.head(10)

,year,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total,feature,commodity
0,2018,1027.0,991.0,1563.0,3581.0,1279.0,1479.0,915.0,3674.0,1497.0,1930.0,NaN,3427.0,NaN,NaN,NaN,10681.0,NaN,dried bananas,banana
1,2017,1047.0,1042.0,988.0,3077.0,866.0,956.0,1058.0,2880.0,1174.0,1317.0,1255.0,3745.0,1473.0,1160.0,1261.0,3894.0,13595.0,dried bananas,banana
2,2016,559.0,1007.0,1211.0,2777.0,1131.0,1134.0,1227.0,3492.0,849.0,902.0,771.0,2521.0,1058.0,1301.0,1045.0,3403.0,12193.0,dried bananas,banana
3,2015,627.0,745.0,1330.0,2703.0,1126.0,1387.0,1220.0,3733.0,1119.0,1103.0,829.0,3051.0,1064.0,756.0,947.0,2767.0,12254.0,dried bananas,banana
4,2014,1084.0,946.0,1161.0,3191.0,1094.0,861.0,1068.0,3023.0,1308.0,1126.0,1059.0,3492.0,708.0,818.0,1180.0,2705.0,12412.0,dried bananas,banana
5,2013,1069.0,996.0,779.0,2844.0,966.0,1329.0,1232.0,3527.0,1241.0,961.0,782.0,2984.0,1063.0,1010.0,1073.0,3145.0,12500.0,dried bananas,banana
6,2012,1029.0,542.0,1042.0,2612.0,864.0,1683.0,1486.0,4033.0,1148.0,1086.0,453.0,2687.0,756.0,904.0,602.0,2261.0,11594.0,dried bananas,banana
7,2011,845.0,736.0,976.0,2557.0,1208.0,1043.0,1694.0,3945.0,1253.0,1140.0,1234.0,3627.0,1186.0,1114.0,914.0,3213.0,13342.0,dried bananas,banana
8,2010,399.0,347.0,394.0,1140.0,592.0,986.0,1049.0,2627.0,1682.0,1279.0,880.0,3841.0,364.0,549.0,885.0,1798.0,9405.0,dried bananas,banana
9,2009,399.0,301.0,433.0,1133.0,661.0,584.0,509.0,1754.0,839.0,675.0,727.0,2241.0,581.0,438.0,404.0,1423.0,6551.0,dried bananas,banana


In [10]:
df.to_csv('import_value_of_bananas_in_thousand_usd.csv', index=False)

df_bananas_value = pd.melt(df, id_vars=['year', 'commodity', 'feature'], 
                     value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

df_bananas_value.head(10)

,year,commodity,feature,variable,value
0,2018,banana,dried bananas,Jan,1027.0
1,2017,banana,dried bananas,Jan,1047.0
2,2016,banana,dried bananas,Jan,559.0
3,2015,banana,dried bananas,Jan,627.0
4,2014,banana,dried bananas,Jan,1084.0
5,2013,banana,dried bananas,Jan,1069.0
6,2012,banana,dried bananas,Jan,1029.0
7,2011,banana,dried bananas,Jan,845.0
8,2010,banana,dried bananas,Jan,399.0
9,2009,banana,dried bananas,Jan,399.0


In [11]:
data = []
filename = './us_import_volume_of_bananas.csv'
with open(filename, 'r') as f:
    for line in f.readlines():
        data.append(line.split('\t'))

data_filtered = []
for idx, item in enumerate(data):
    if item[-1]=='\n':
        item.pop(-1)
    item[-1] = item[-1].strip()
    if idx!=0:
        item = [item[0]] + [float(e.strip().replace(',', '')) if e!='NA' else None for e in item[1:] if len(e.strip())>0]
    else:
        item = [e.strip() for e in item if len(e.strip())>0 ]
    data_filtered.append(item)
    
df = pd.DataFrame(data_filtered[1:], columns=data_filtered[0])    
df['feature'] = df.index.map(lambda idx: assign_feature(idx))   
df['commodity'] = 'banana'
df.head(10)

,year,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total,feature,commodity
0,2018,626.0,539.0,1160.0,2325.0,775.0,809.0,491.0,2075.0,961.0,1248.0,NaN,2209.0,NaN,NaN,NaN,NaN,6609.0,dried bananas,banana
1,2017,525.0,546.0,639.0,1710.0,450.0,570.0,635.0,1656.0,737.0,708.0,828.0,2273.0,895.0,651.0,782.0,2327.0,7966.0,dried bananas,banana
2,2016,203.0,570.0,758.0,1531.0,781.0,754.0,789.0,2324.0,560.0,487.0,508.0,1556.0,727.0,779.0,477.0,1983.0,7394.0,dried bananas,banana
3,2015,301.0,466.0,623.0,1390.0,785.0,896.0,938.0,2619.0,857.0,671.0,435.0,1962.0,689.0,435.0,477.0,1600.0,7572.0,dried bananas,banana
4,2014,691.0,662.0,793.0,2146.0,889.0,644.0,801.0,2335.0,706.0,696.0,641.0,2043.0,378.0,418.0,725.0,1522.0,8045.0,dried bananas,banana
5,2018,875710.0,835487.0,887093.0,2598290.0,886664.0,998492.0,867427.0,2752583.0,953709.0,894495.0,NaN,1848204.0,NaN,NaN,NaN,NaN,7199077.0,dried bananas,banana
6,2017,848217.0,837063.0,971044.0,2656324.0,808934.0,1029832.0,920192.0,2758958.0,835774.0,908539.0,897241.0,2641554.0,900097.0,910932.0,738191.0,2549221.0,10606056.0,dried bananas,banana
7,2016,788484.0,850755.0,957557.0,2596797.0,847570.0,915754.0,890370.0,2653695.0,749078.0,931343.0,815962.0,2496384.0,795253.0,811086.0,773624.0,2379963.0,10126837.0,dried bananas,banana
8,2015,772765.0,818554.0,946356.0,2537676.0,886909.0,911054.0,848876.0,2646839.0,832566.0,842433.0,910572.0,2585571.0,833261.0,794203.0,807809.0,2435273.0,10205359.0,dried bananas,banana
9,2014,807093.0,797134.0,843222.0,2447449.0,904920.0,901380.0,882944.0,2689244.0,866594.0,808114.0,866814.0,2541522.0,855581.0,704918.0,863406.0,2423905.0,10102121.0,dried bananas,banana


In [12]:
df.to_csv('import_volume_of_bananas_in_thousand_pounds.csv', index=False)

df_bananas_volume = pd.melt(df, id_vars=['year', 'feature', 'commodity'], 
                     value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])


df_bananas_volume.head(10)

,year,feature,commodity,variable,value
0,2018,dried bananas,banana,Jan,626.0
1,2017,dried bananas,banana,Jan,525.0
2,2016,dried bananas,banana,Jan,203.0
3,2015,dried bananas,banana,Jan,301.0
4,2014,dried bananas,banana,Jan,691.0
5,2018,dried bananas,banana,Jan,875710.0
6,2017,dried bananas,banana,Jan,848217.0
7,2016,dried bananas,banana,Jan,788484.0
8,2015,dried bananas,banana,Jan,772765.0
9,2014,dried bananas,banana,Jan,807093.0


In [13]:
df_bananas = pd.merge(left=df_bananas_value, 
                      right=df_bananas_volume, 
                      on=['year', 'feature', 'variable', 'commodity'], 
                      suffixes=['_bnn_1000_usd', '_bnn_1000_pounds'])
df_bananas['timestamp'] = df_bananas[['variable', 'year']].apply(lambda row: row['variable'] + ', ' + row['year'], axis=1)
df_bananas.head(10)

,year,commodity,feature,variable,value_bnn_1000_usd,value_bnn_1000_pounds,timestamp
0,2018,banana,dried bananas,Jan,1027.0,626.0,"Jan, 2018"
1,2018,banana,dried bananas,Jan,1027.0,875710.0,"Jan, 2018"
2,2017,banana,dried bananas,Jan,1047.0,525.0,"Jan, 2017"
3,2017,banana,dried bananas,Jan,1047.0,848217.0,"Jan, 2017"
4,2016,banana,dried bananas,Jan,559.0,203.0,"Jan, 2016"
5,2016,banana,dried bananas,Jan,559.0,788484.0,"Jan, 2016"
6,2015,banana,dried bananas,Jan,627.0,301.0,"Jan, 2015"
7,2015,banana,dried bananas,Jan,627.0,772765.0,"Jan, 2015"
8,2014,banana,dried bananas,Jan,1084.0,691.0,"Jan, 2014"
9,2014,banana,dried bananas,Jan,1084.0,807093.0,"Jan, 2014"


# Import value and volume of Bananas from USDA

```
https://data.ers.usda.gov/reports.aspx?programArea=fruit&stat_year=2009&top=5&HardCopy=True&RowsPerPage=25&groupName=Citrus&commodityName=Oranges&ID=17851
```

In [14]:
def assign_feature(idx):
    if idx < 4:
        return 'fresh oranges'
    elif idx < 8:
        return 'orange juice'
    else:
        return 'preserved oranges'

In [15]:
import pandas as pd

data = []
filename = './us_import_value_of_oranges.csv'
with open(filename, 'r') as f:
    for line in f.readlines():
        data.append([e for e in line.split('\t') if len(e.strip())>0])

data_filtered = []
for idx, item in enumerate(data):
    if item[-1]=='\n':
        item.pop(-1)
    item[-1] = item[-1].strip()
    if idx!=0:
        item = [item[0].strip()] + [float(e.strip().replace(',', '')) if e!='NA' else None for e in item[1:]]
    else:
        item = [e.strip() for e in item]
    data_filtered.append([e for e in item])
    
    
df = pd.DataFrame(data_filtered[1:], columns=data_filtered[0])


    
df['feature'] = df.index.map(lambda idx: assign_feature(idx))  
df['commodity'] = 'orange'
df.head(10)

,year,Nov,Dec,Jan,Q1,Feb,Mar,Apr,Q2,May,Jun,Jul,Q3,Aug,Sep,Oct,Q4,Total,feature,commodity
0,2017/18,2480.0,3918.0,4436.0,10834.0,4806.0,7068.0,7774.0,19648.0,13115.0,12512.0,54828.0,80455.0,54933.0,NaN,NaN,54933.0,165870.0,fresh oranges,orange
1,2016/17,1239.0,1689.0,2444.0,5372.0,3266.0,5433.0,5136.0,13834.0,7550.0,14372.0,43110.0,65031.0,42323.0,16488.0,16747.0,75558.0,159795.0,fresh oranges,orange
2,2015/16,1415.0,2158.0,2575.0,6147.0,2984.0,3415.0,4133.0,10532.0,3817.0,6247.0,35690.0,45754.0,47530.0,24762.0,6014.0,78306.0,140738.0,fresh oranges,orange
3,2014/15,932.0,1618.0,2417.0,4968.0,2587.0,3040.0,3801.0,9428.0,4406.0,8183.0,38935.0,51524.0,37455.0,27124.0,8895.0,73473.0,139393.0,fresh oranges,orange
4,2017/18,79215.0,50131.0,82891.0,212237.0,78866.0,105652.0,121341.0,305859.0,104051.0,108588.0,65584.0,278223.0,46496.0,30829.0,NaN,77325.0,873643.0,orange juice,orange
5,2016/17,66737.0,34928.0,79315.0,180980.0,66343.0,93798.0,85089.0,245231.0,102496.0,77418.0,36254.0,216168.0,39182.0,38732.0,50668.0,128582.0,770961.0,orange juice,orange
6,2015/16,25326.0,35912.0,37113.0,98351.0,48691.0,71318.0,83675.0,203684.0,68323.0,47778.0,31993.0,148095.0,32293.0,39198.0,31741.0,103231.0,553362.0,orange juice,orange
7,2014/15,68604.0,45823.0,61320.0,175746.0,47368.0,43817.0,118856.0,210040.0,49221.0,81862.0,24958.0,156042.0,16591.0,26066.0,28975.0,71633.0,613461.0,orange juice,orange
8,2017/18,1353.0,867.0,1054.0,3273.0,1323.0,1243.0,1822.0,4389.0,1525.0,1470.0,1319.0,4314.0,1077.0,NaN,NaN,1077.0,13052.0,preserved oranges,orange
9,2016/17,1021.0,1257.0,998.0,3276.0,808.0,706.0,619.0,2134.0,698.0,673.0,714.0,2085.0,755.0,728.0,915.0,2399.0,9894.0,preserved oranges,orange


In [16]:
df.to_csv('import_value_of_oranges_in_thousand_usd.csv', index=False)

df_oranges_value = pd.melt(
                     df, 
                     id_vars=['year', 'feature', 'commodity'], 
                     value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec'])
df_oranges_value.year = df_oranges_value.year.map(lambda x: x.split('/')[0])
df_oranges_value.head(3)

,year,feature,commodity,variable,value
0,2017,fresh oranges,orange,Jan,4436.0
1,2016,fresh oranges,orange,Jan,2444.0
2,2015,fresh oranges,orange,Jan,2575.0


In [17]:
import pandas as pd

data = []
filename = './us_import_volume_of_oranges.csv'
with open(filename, 'r') as f:
    for line in f.readlines():
        data.append([e for e in line.split('\t') if len(e.strip())>0])

data_filtered = []
for idx, item in enumerate(data):
    if item[-1]=='\n':
        item.pop(-1)
    item[-1] = item[-1].strip()
    if idx!=0:
        item = [item[0].strip()] + [float(e.strip().replace(',', '')) if e!='NA' else None for e in item[1:]]
    else:
        item = [e.strip() for e in item]
    data_filtered.append([e for e in item])
    
    
df = pd.DataFrame(data_filtered[1:], columns=data_filtered[0])    
df['feature'] = df.index.map(lambda idx: assign_feature(idx))   
df['commodity'] = 'orange'
df.head(3)    

,year,Nov,Dec,Jan,Q1,Feb,Mar,Apr,Q2,May,Jun,Jul,Q3,Aug,Sep,Oct,Q4,Total,feature,commodity
0,2017/18,9335.0,14663.0,14504.0,38502.0,16735.0,24777.0,27749.0,69260.0,35818.0,27192.0,113645.0,176655.0,116401.0,NaN,NaN,116401.0,400818.0,fresh oranges,orange
1,2016/17,5454.0,9070.0,12385.0,26909.0,15381.0,21158.0,18436.0,54975.0,28596.0,38176.0,93048.0,159820.0,90529.0,33920.0,36163.0,160612.0,402315.0,fresh oranges,orange
2,2015/16,8401.0,12345.0,12935.0,33681.0,16059.0,17184.0,18347.0,51590.0,16872.0,17710.0,75647.0,110229.0,101206.0,51344.0,13619.0,166169.0,361670.0,fresh oranges,orange


In [18]:
df.to_csv('import_volume_of_oranges_in_thousand_usd.csv', index=False)

df_oranges_volume = pd.melt(
                    df, 
                     id_vars=['year', 'feature', 'commodity'], 
                     value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec'])
df_oranges_volume.year = df_oranges_volume.year.map(lambda x: x.split('/')[0])
df_oranges_volume.head(3)

,year,feature,commodity,variable,value
0,2017,fresh oranges,orange,Jan,14504.0
1,2016,fresh oranges,orange,Jan,12385.0
2,2015,fresh oranges,orange,Jan,12935.0


In [20]:
df_oranges = pd.merge(left=df_oranges_value, 
                      right=df_oranges_volume, 
                      on=['year', 'variable', 'feature', 'commodity'], 
                      suffixes=['_orng_1000_usd', '_orng_1000_pounds'])
df_oranges['timestamp'] = df_oranges[['variable', 'year']].apply(lambda row: row['variable'] + ', ' + row['year'], axis=1)
df_oranges.head(10)

,year,feature,commodity,variable,value_orng_1000_usd,value_orng_1000_pounds,timestamp
0,2017,fresh oranges,orange,Jan,4436.0,14504.0,"Jan, 2017"
1,2016,fresh oranges,orange,Jan,2444.0,12385.0,"Jan, 2016"
2,2015,fresh oranges,orange,Jan,2575.0,12935.0,"Jan, 2015"
3,2014,fresh oranges,orange,Jan,2417.0,12001.0,"Jan, 2014"
4,2017,orange juice,orange,Jan,82891.0,52970.0,"Jan, 2017"
5,2016,orange juice,orange,Jan,79315.0,47110.0,"Jan, 2016"
6,2015,orange juice,orange,Jan,37113.0,26280.0,"Jan, 2015"
7,2014,orange juice,orange,Jan,61320.0,44496.0,"Jan, 2014"
8,2017,preserved oranges,orange,Jan,1054.0,1161.0,"Jan, 2017"
9,2016,preserved oranges,orange,Jan,998.0,1153.0,"Jan, 2016"


# Merge Bananas and Oranges, Import Value and Volume

In [21]:
df_merged = pd.merge(left=df_bananas, right=df_oranges, on=['timestamp', 'variable', 'year'])

In [22]:
df_merged.head(10)

,year,commodity_x,feature_x,variable,value_bnn_1000_usd,value_bnn_1000_pounds,timestamp,feature_y,commodity_y,value_orng_1000_usd,value_orng_1000_pounds
0,2017,banana,dried bananas,Jan,1047.0,525.0,"Jan, 2017",fresh oranges,orange,4436.0,14504.0
1,2017,banana,dried bananas,Jan,1047.0,525.0,"Jan, 2017",orange juice,orange,82891.0,52970.0
2,2017,banana,dried bananas,Jan,1047.0,525.0,"Jan, 2017",preserved oranges,orange,1054.0,1161.0
3,2017,banana,dried bananas,Jan,1047.0,848217.0,"Jan, 2017",fresh oranges,orange,4436.0,14504.0
4,2017,banana,dried bananas,Jan,1047.0,848217.0,"Jan, 2017",orange juice,orange,82891.0,52970.0
5,2017,banana,dried bananas,Jan,1047.0,848217.0,"Jan, 2017",preserved oranges,orange,1054.0,1161.0
6,2017,banana,fresh bananas,Jan,169654.0,4344.0,"Jan, 2017",fresh oranges,orange,4436.0,14504.0
7,2017,banana,fresh bananas,Jan,169654.0,4344.0,"Jan, 2017",orange juice,orange,82891.0,52970.0
8,2017,banana,fresh bananas,Jan,169654.0,4344.0,"Jan, 2017",preserved oranges,orange,1054.0,1161.0
9,2017,banana,fresh bananas,Jan,169654.0,24468.0,"Jan, 2017",fresh oranges,orange,4436.0,14504.0


## Save second dataset

In [24]:
df_merged.to_csv('usda_oranges_and_bananas_data.csv', index=False)